In [1]:
from IPython.core.display import HTML, display
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# Specify the path to the fastai directory
lib_PATH = '/home/ubuntu/fastai/fastai'   # Update this
!cd {lib_PATH}
# Create sym-link to the fastai library
!ln -s {lib_PATH} ./

ln: failed to create symbolic link './fastai': File exists


In [4]:
# Load required modules
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from dateutil.relativedelta import relativedelta

# Load Cleaned Data

In [35]:
train = pd.read_feather('/home/ubuntu/wsmd_music_rec/data/tmp/wsdm_train')
test = pd.read_feather('/home/ubuntu/wsmd_music_rec/data/tmp/wsdm_test')

In [36]:
test.song_year.replace([np.inf, -np.inf], np.nan, inplace=True)
test.song_year.fillna(-1,inplace=True)
test.song_year = test.song_year.astype('int64')

In [ ]:
train.columns[train.isnull().any()]

In [6]:
df, y, nas = proc_df(train, 'target')    # proc_df deals with na values

In [ ]:
df.columns[df.isnull().any()]     # no nas left

# Split into Training + Validation Sets

In [ ]:
fig=plt.figure(figsize=(18, 16))
test.song_year.value_counts().plot.barh()

In [ ]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()
# https://www.kaggle.com/kamilkk/i-have-to-say-this
# temporally ordered data - set val. set = test set's length
len_valid = len(test)
n_trn = len(df) - len_valid
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

# Train the Random Forest

In [10]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    auc_trn   = roc_auc_score(y_train, m.predict(X_train))
    auc_valid = roc_auc_score(y_valid, m.predict(X_valid))
    res = [auc_trn, auc_valid, rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid), 
           m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print('auc_trn, auc_valid, RMSE - train, RMSE - valid, R^2 - train, R^2 - valid')
    print(res)
    
def print_score_final(m):
    auc_trn = roc_auc_score(y, m.predict(df))
    res = [auc_trn, rmse(m.predict(df), y), m.score(df, y)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print('auc_trn, RMSE - train, R^2 - train')
    print(res)

In [ ]:
m = RandomForestClassifier(n_estimators=50, min_samples_leaf=20, n_jobs=-1, verbose=1)
%time m.fit(X_train, y_train)
print_score(m)

# Retrain Model Using All Data

In [ ]:
m_final = RandomForestClassifier(n_estimators=50, min_samples_leaf=20, n_jobs=-1, verbose=1)
%time m_final.fit(df, y)

In [11]:
print_score_final(m_final)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   25.5s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:   34.1s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   25.2s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:   34.7s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   25.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:   34.4s finished


auc_trn, RMSE - train, R^2 - train
[0.75533052178863902, 0.4944190111418818, 0.75554984142148374]


# Make Predictions

In [ ]:
def add_dum_col(df):
    df['dummy'] = 1
    df_new, _, _ = proc_df(df, 'dummy')
    return df_new

test_num = add_dum_col(test)   # Numericalize grocery_test dataframe

In [50]:
test_df, _, _ = proc_df(test, 'id')

In [51]:
%time predictions = m_final.predict(test_df)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    6.3s


CPU times: user 1min 3s, sys: 940 ms, total: 1min 4s
Wall time: 9.49 s


[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    8.7s finished


In [59]:
output = pd.concat([test['id'], pd.DataFrame(predictions)], axis=1)
output.columns = ['id', 'target']   #Changing column names
output.to_csv('predicted.csv', index=False)